In [1]:
import os
from langchain_community.embeddings import OllamaEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import UnstructuredEPubLoader

embeddings = OllamaEmbeddings(model='nomic-embed-text:v1.5')

# sudo apt-get install pandoc

In [1]:
from pypandoc.pandoc_download import download_pandoc
# see the documentation how to customize the installation path
# but be aware that you then need to include it in the `PATH`
#download_pandoc()

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/cristian/Documentos/Projetos/personal-search/ingestion/livros/Conselhos para obreiros - Charles H Spurgeon.pdf")
documents = loader.load()

In [4]:
loader = UnstructuredEPubLoader("/home/cristian/Documentos/Projetos/personal-search/ingestion/livros/Livro Conselhos para obreiros - Charles Spurgeon.epub", mode="elements")
data = loader.load()

documents = loader.load()

19


In [5]:
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100
)

chunked_docs = text_spliter.split_documents(documents)


db = FAISS.from_documents(chunked_docs, embeddings)
db.save_local("../vector-store/books")
print(db.index.ntotal)

151


In [ ]:
#db.add_documents(chunked_docs)

In [6]:
#testar se o documento foi adicionado
db2 = FAISS.load_local("../vector-store/books", embeddings, allow_dangerous_deserialization=True)
retriever = db2.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 3

results = retriever.get_relevant_documents("Calvinismo e livre-arbítrio")

In [7]:
results

[Document(page_content='7\nConforme a vossa fé\nada é impossível ao homem que sabe como conquistar o céu por meio da intercessão\ntenaz. Quando vemos um, dois, dez ou vinte penitentes convertidos, e quando, por\nvezes, nos sentimos profundamente gratos porque uma centena deles foi acrescentada\nà igreja em um mês, devemos ficar satisfeitos? Será que não deveríamos pensar que a oração que\nfoi abençoada com a conversão de cem pessoas, se tivesse sido mais fervorosa, poderia, dentro\ndo propósito divino, ser respondida com a conversão de mil? Por que não? Eu não sei por que\nLondres  não poderia ser sacudida de ponta a ponta com a verdade do evangelho antes de\ncompletados doze meses, a partir deste dia. Vocês dirão: “Não temos ministros suficientes”. Mas\nDeus pode produzi-los. Ele pode encontrar ministros para pregarem sua verdade — ah, se ele\nquiser, em meio à própria escória da terra. Ele pode tomar o pior dos homens, o mais vil dos vis,\ntransformar seu coração e torná-lo pregador 